# Load the datasets

The path to dataset directory and patterns to search in those directories for the HPC, PFC recordings are loaded from the config file.

In [2]:
from phasic_tonic.detect_phasic import detect_phasic
from phasic_tonic.DatasetLoader import DatasetLoader
from phasic_tonic.helper import get_metadata
from phasic_tonic.runtime_logger import logger_setup    
from phasic_tonic.utils import get_sequences, get_start_end, preprocess

import numpy as np
import pandas as pd
import pynapple as nap

from tqdm.auto import tqdm
from scipy.io import loadmat

fs_cbd = 2500
fs_os = 2500
fs_rgs = 1000

targetFs = 500
n_down_cbd = fs_cbd/targetFs
n_down_rgs = fs_rgs/targetFs
n_down_os = fs_os/targetFs

logger = logger_setup()

CONFIG_DIR = "/home/nero/phasic_tonic/data/dataset_loading.yaml"

Datasets = DatasetLoader(CONFIG_DIR)
mapped_datasets = Datasets.load_datasets()

Check the number of loaded recordings

In [3]:
cbd_cnt = 0
rgs_cnt = 0
os_cnt = 0

# Count recordings belonging to CBD dataset
for name in mapped_datasets:
    metadata = get_metadata(name)
    if metadata['treatment'] == 0 or metadata['treatment'] == 1:
        cbd_cnt += 1
    elif metadata['treatment'] == 2 or metadata['treatment'] == 3:
        rgs_cnt += 1
    elif metadata['treatment'] == 4:
        os_cnt += 1

assert cbd_cnt == 170
assert rgs_cnt == 159
assert os_cnt == 210

In [4]:
mapped_datasets["Rat9_SD6_OD_2_posttrial4"]

('/home/nero/datasets/RGS14/StableCondOD/OS_Ephys_RGS14_Rat9_378134_SD6_OD_15-16_05_2020/2020-05-15_14-43-40_Post_Trial4/2020-05-15_14-43-40_Post_Trial4-states.mat',
 '/home/nero/datasets/RGS14/StableCondOD/OS_Ephys_RGS14_Rat9_378134_SD6_OD_15-16_05_2020/2020-05-15_14-43-40_Post_Trial4/HPC_100_CH9_0.continuous.mat',
 '/home/nero/datasets/RGS14/StableCondOD/OS_Ephys_RGS14_Rat9_378134_SD6_OD_15-16_05_2020/2020-05-15_14-43-40_Post_Trial4/PFC_100_CH26_0.continuous.mat')

# Loop through the dataset

In [6]:
with tqdm(mapped_datasets) as mapped_tqdm:
    for name in mapped_tqdm:
        metadata = get_metadata(name)
        mapped_tqdm.set_postfix_str(name)
        states_fname, hpc_fname, pfc_fname = mapped_datasets[name]
        logger.debug("Loading: {0}".format(name))

        if metadata["treatment"] == 0 or metadata["treatment"] == 1:
            n_down = n_down_cbd
        elif metadata["treatment"] == 2 or metadata["treatment"] == 3:
            n_down = n_down_rgs
        elif metadata["treatment"] == 4:
            n_down = n_down_os
        
        # Load the LFP data
        lfpHPC = loadmat(hpc_fname)['HPC'].flatten()
        lfpPFC = loadmat(pfc_fname)['PFC'].flatten()

        # Load the states
        hypno = loadmat(states_fname)['states'].flatten()
        
        # Skip if no REM epoch is detected
        if(not (np.any(hypno == 5))):
            logger.debug("No REM detected. Skipping.")
            continue
        elif(np.sum(np.diff(get_sequences(np.where(hypno == 5)[0]))) < 10):
            logger.debug("No REM longer than 10s. Skipping.")
            continue
        
        # Create Pynapple IntervalSet        
        start, end = get_start_end(hypno, sleep_state_id=5)
        rem_interval = nap.IntervalSet(start=start, end=end)
        
        # Create TsdFrame for HPC and PFC signals
        fs = n_down*targetFs
        t = np.arange(0, len(lfpHPC)/fs, 1/fs)
        lfp = nap.TsdFrame(t=t, d=np.vstack([lfpHPC, lfpPFC]).T, columns=['HPC', 'PFC'])
        
        # Detect phasic intervals
        lfpHPC_down = preprocess(lfpHPC, n_down)
        phREM = detect_phasic(lfpHPC_down, hypno, targetFs)
        
        # Create phasic REM IntervalSet
        start, end = [], []
        for rem_idx in phREM:
            for s, e in phREM[rem_idx]:
                start.append(s/targetFs)
                end.append(e/targetFs)
        phasic_interval = nap.IntervalSet(start, end)
        tonic_interval = rem_interval.set_diff(phasic_interval)
        break

  0%|          | 0/539 [00:00<?, ?it/s]

## Access the HPC and PFC signals during phasic REM

In [8]:
lfp.restrict(phasic_interval)

Time (s)         HPC         PFC
----------  --------  ----------
1379.984    -226.951    37.5347
1379.9844   -217.58     59.8681
1379.9848   -198.255    75.2631
1379.9852   -206.673    78.1636
1379.9856   -193.815   107.547
1379.986    -156.379   168.706
1379.9864   -206.15    118.607
...
2432.3056   -220.943   -27.4514
2432.306    -206.43    -13.7936
2432.3064   -235.411   -31.6098
2432.3068   -273.278   -77.5986
2432.3072   -283.825   -80.6341
2432.3076   -203.913     6.07146
2432.308    -315.641  -113.657
dtype: float64, shape: (22574, 2)

In [9]:
phrem_hpc = [lfp["HPC"].restrict(phasic_interval[i]).to_numpy() for i in range(len(phasic_interval))]
tonic_hpc = [lfp["HPC"].restrict(tonic_interval[i]).to_numpy() for i in range(len(tonic_interval))]

In [25]:
np.savez(name+'_phasic', *phrem_hpc)
np.savez(name+'_tonic', *tonic_hpc)

## Access HPC and PFC signals during tonic REM

In [11]:
lfp.restrict(tonic_interval)

Time (s)           HPC       PFC
----------  ----------  --------
1331.0      -180.893    -93.2364
1331.0004   -150.785    -69.5545
1331.0008   -132.412    -70.9175
1331.0012    -85.3764   -50.1542
1331.0016    -79.2403   -50.5931
1331.002     -75.6617   -49.3135
1331.0024    -57.6323   -34.4635
...
2474.9976   -125.165     76.2955
2474.998    -118.796     64.7644
2474.9984     -4.49443  173.913
2474.9988    -37.3338   133.221
2474.9992   -137.777     25.3803
2474.9996   -103.864     80.9616
2475.0       -36.7785   134.176
dtype: float64, shape: (479936, 2)